1. 2017data 아래에 2018데이터 이어붙여서 ('Code' 기준으로 정렬 후, 'Date'로 정렬)
2. M_lst먼저 만들고
3. cci만들고
4. TV filtering

# Library

In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# stock1718_clean.csv

In [2]:
s18 = pd.read_csv('stock_2018.csv')
s17_re = pd.read_csv('s17_re.csv')

In [3]:
## 여기 하고 del_c_nms는 건너뛰고 M 만들고나서 바로 인터넷 코드로 적용해보기 쪽으로 넘어갈 것.
s17_re.columns

s_all = pd.concat([s17_re,s18[s17_re.columns]])
s_all = s_all.sort_values(['Code', 'Date']).reset_index(drop = True)[['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change']] # code, date로 정렬
s_all

Index(['Date', 'Code', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change'], dtype='object')

,Code,Date,Open,High,Low,Close,Volume,Change
0,20,2017-11-01,8850,9170,8840,9170,115335,0.037330
1,20,2017-11-02,9170,9180,9030,9100,68630,-0.007634
2,20,2017-11-03,9080,9270,9070,9240,117255,0.015385
3,20,2017-11-06,9300,9500,9270,9490,369895,0.027056
4,20,2017-11-07,9500,9530,9260,9340,166998,-0.015806
...,...,...,...,...,...,...,...,...
461749,950160,2017-12-21,44650,46150,43500,43500,1077534,-0.003436
461750,950160,2017-12-22,44350,45250,43600,44500,816742,0.022989
461751,950160,2017-12-26,44200,46800,44200,45250,960367,0.016854
461752,950160,2017-12-27,46600,49500,46100,49500,1324792,0.093923


## del_c_nms : 2017이 마지막 기록인 code 모음
* 아래 3개 cell은 한 번 실행 안해보기.
* 구글링 해서 얻은 cci 구하는 코드로 cci 구할 때도 아래 3개 cell 실행 안시키기!!!!!!!

In [ ]:
del_c_nms = s_all['Code'].unique()[s_all.groupby('Code')['Date'].max().str.contains('2017-')==1]

In [ ]:
# del_c 들의 row구해서 총 몇 row가 빠지는지 계산해보기.
len(del_c_nms)
del_c_nms[0]
len(del_c_nms) * s_all[s_all['Code'] == 1290].shape[0] # 이거 아님. 모든 종목의 일 수가 41일은 아니므로
# s_all[s_all['Code'] == 1290]

rows = 0
for i in del_c_nms:
    if s_all[s_all['Code'] == i].shape[0] != 41:
        rows += s_all[s_all['Code'] == i].shape[0]
        i, s_all[s_all['Code'] == i].shape[0]
    else:
        rows += s_all[s_all['Code'] == i].shape[0]
rows # 총 17205개의 row가 빠짐.

In [ ]:
for i in del_c_nms:
    s_all = s_all.drop(s_all[s_all['Code'] == i].index, axis = 0)
s_all

In [4]:
s_all.reset_index(drop = True, inplace = True)

# M = (High+Low+Close)/3
얘까지 하고나면 인터넷 코드로 적용해보기로 넘어가기

In [5]:
s_all['M'] = s_all[['High', 'Low', 'Close']].sum(axis = 1)/3
s_all

,Code,Date,Open,High,Low,Close,Volume,Change,M
0,20,2017-11-01,8850,9170,8840,9170,115335,0.037330,9060.000000
1,20,2017-11-02,9170,9180,9030,9100,68630,-0.007634,9103.333333
2,20,2017-11-03,9080,9270,9070,9240,117255,0.015385,9193.333333
3,20,2017-11-06,9300,9500,9270,9490,369895,0.027056,9420.000000
4,20,2017-11-07,9500,9530,9260,9340,166998,-0.015806,9376.666667
...,...,...,...,...,...,...,...,...,...
461749,950160,2017-12-21,44650,46150,43500,43500,1077534,-0.003436,44383.333333
461750,950160,2017-12-22,44350,45250,43600,44500,816742,0.022989,44450.000000
461751,950160,2017-12-26,44200,46800,44200,45250,960367,0.016854,45416.666667
461752,950160,2017-12-27,46600,49500,46100,49500,1324792,0.093923,48366.666667


# m = M의 N기간 동안의 단순 이동평균
M을 20일 씩 묶어서 평균내기

In [6]:
from tqdm import tqdm
N = 20
m_all_dic = {}

for i in tqdm(s_all['Code'].unique()):
#     while s_all[s_all['Code']==i]:
    tmp = s_all[s_all['Code'] == i]
    m_lst = [np.NaN] * 19 # 20일씩 이동평균이므로 앞에 NaN이 19개가 존재하게 됨.

    for j in range(len(tmp)):
        if j > len(tmp)-N: # 길이가 N이 넘어가면 왜 pass..?  님... N일 이하로 남았을 때라는 뜻이잖아요...
                            # 이 옆엔 바보같은 생각...2N, 3N...도 있을 수도 있잖아.. 나눴을 때 type이 float 형이면, 그 때 몫*N만큼을 뺀 길이가 N 미만이면 pass
#             m_lst = [] # 초기화... 빈 list여야 오류 안남. # 이러면 빈 리스트만 들어가게 됨;;;
            pass
        else:
            m = tmp[['M']][j:j+N].values.sum()/N
            m_lst.append(m)
    m_all_dic[i] = m_lst

100%|██████████| 1993/1993 [01:59<00:00, 16.62it/s] 


In [7]:
len(m_all_dic)

m_all_lst = []
for i in m_all_dic.keys():
    m_all_lst.extend(m_all_dic[i])
    
len(m_all_lst) #444549면 되는 것.


1993

461850

## s_all ('stock1718_clean.csv') 옆에 m_all_lst 붙이기

In [8]:
m_sr = pd.Series(m_all_lst, name = 'm')
m_df = pd.concat([s_all, m_sr], axis = 1)

# d = 

## abs(M-m)

In [9]:
m_df['abs(M-m)'] = abs(m_df['M']-m_df['m'])

In [10]:
m_df # 444549

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,abs(M-m)
0,20.0,2017-11-01,8850.0,9170.0,8840.0,9170.0,115335.0,0.037330,9060.000000,NaN,NaN
1,20.0,2017-11-02,9170.0,9180.0,9030.0,9100.0,68630.0,-0.007634,9103.333333,NaN,NaN
2,20.0,2017-11-03,9080.0,9270.0,9070.0,9240.0,117255.0,0.015385,9193.333333,NaN,NaN
3,20.0,2017-11-06,9300.0,9500.0,9270.0,9490.0,369895.0,0.027056,9420.000000,NaN,NaN
4,20.0,2017-11-07,9500.0,9530.0,9260.0,9340.0,166998.0,-0.015806,9376.666667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
461845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51690.000000,NaN
461846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50799.166667,NaN
461847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50136.666667,NaN
461848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49821.666667,NaN


## abs(M-m) == NaN이면 지우고 reset_index(drop = True)하기

In [11]:
m_df = m_df.drop(m_df[pd.isnull(m_df['abs(M-m)'])==True].index)

In [12]:
m_df.reset_index(drop = True, inplace = True)
# m_df.drop('index', axis = 1, inplace = True) # reset_index하고 drop = True 놓쳤을 때
m_df # 414890

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,abs(M-m)
0,20.0,2017-11-28,10950.0,11050.0,10700.0,10750.0,146968.0,-0.022727,10833.333333,10372.666667,460.666667
1,20.0,2017-11-29,10800.0,10950.0,10600.0,10750.0,196725.0,0.000000,10766.666667,10458.000000,308.666667
2,20.0,2017-11-30,10700.0,10800.0,10300.0,10300.0,336283.0,-0.041860,10466.666667,10526.166667,59.500000
3,20.0,2017-12-01,10450.0,10650.0,10350.0,10550.0,216633.0,0.024272,10516.666667,10592.333333,75.666667
4,20.0,2017-12-04,10600.0,10750.0,10250.0,10350.0,228353.0,-0.018957,10450.000000,10643.833333,193.833333
...,...,...,...,...,...,...,...,...,...,...,...
423920,950160.0,2017-12-21,44650.0,46150.0,43500.0,43500.0,1077534.0,-0.003436,44383.333333,4106.666667,40276.666667
423921,950160.0,2017-12-22,44350.0,45250.0,43600.0,44500.0,816742.0,0.022989,44450.000000,4108.666667,40341.333333
423922,950160.0,2017-12-26,44200.0,46800.0,44200.0,45250.0,960367.0,0.016854,45416.666667,4111.416667,41305.250000
423923,950160.0,2017-12-27,46600.0,49500.0,46100.0,49500.0,1324792.0,0.093923,48366.666667,4118.166667,44248.500000


## d = abs(M-m)을 N일씩 평균

In [ ]:
from tqdm import tqdm

N = 20
d_all_dic = {}

for i in tqdm(m_df['Code'].unique()):
    tmp = m_df[m_df['Code'] == i]
    d_lst = [np.NaN] * 19 # 20일씩 이동평균이므로 앞에 NaN이 19개가 존재하게 됨.

    for j in range(len(tmp)):
        if j > len(tmp)-N: # 길이가 N이 넘어가면 왜 pass..?  님... N일 이하로 남았을 때라는 뜻이잖아요...
                            # 이 옆엔 바보같은 생각...2N, 3N...도 있을 수도 있잖아.. 나눴을 때 type이 float 형이면, 그 때 몫*N만큼을 뺀 길이가 N 미만이면 pass
#             d_lst = [] # 초기화... 빈 list여야 오류 안남. # 이러면 빈 리스트만 들어가게 됨;;;
            pass
        else:
            d = tmp[['abs(M-m)']][j:j+N].values.sum()/N
            d_lst.append(d)
    d_all_dic[i] = d_lst

 89%|████████▉ | 1766/1989 [01:46<00:11, 19.90it/s]

In [ ]:
len(d_all_dic)

d_all_lst = []
for i in d_all_dic.keys():
    d_all_lst.extend(d_all_dic[i])
    
len(d_all_lst) # 414890

In [ ]:
d_all_lst

## m_df ('m_df.csv')의 abs(M-m) 열 옆에 d_all_lst 붙이기

In [ ]:
len(m_df)
m_df # 414890

In [ ]:
d_sr = pd.Series(d_all_lst, name = 'd')
d_sr
d_df = pd.concat([m_df, d_sr], axis = 1)
d_df # 414890

## NaN 들어있는 행 지우기

In [ ]:
d_df = d_df.drop(d_df[pd.isnull(d_df['d'])==True].index)

d_df.reset_index(drop=True, inplace = True)
d_df # 385231

In [ ]:
d_df['Code'].unique()
d_df = d_df[d_df['Date'].str.contains('2017-')==0] # 2017-이 안들어있는 것만. 즉, 2018만
d_df.reset_index(drop = True, inplace = True)
d_df # 380551

# CCI=(M-m)/(0.015*d)

In [ ]:
d_df['CCI'] = (d_df['M'] - d_df['m']) / (0.015 * d_df['d'])

In [22]:
d_df # 380551
d_df['Code'].unique()
d_df['Code'].nunique()

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,abs(M-m),d,CCI
0,20.0,2018-01-02,9750.0,9900.0,9700.0,9870.0,120676.0,0.012308,9823.333333,9819.666667,3.666667,566.708333,0.431341
1,20.0,2018-01-03,9900.0,10250.0,9820.0,10000.0,268220.0,0.013171,10023.333333,9795.000000,228.333333,574.341667,26.503775
2,20.0,2018-01-04,10050.0,10050.0,9680.0,9750.0,161342.0,-0.025000,9826.666667,9763.833333,62.833333,567.791667,7.377510
3,20.0,2018-01-05,9750.0,9980.0,9750.0,9910.0,116604.0,0.016410,9880.000000,9744.500000,135.500000,553.483333,16.320877
4,20.0,2018-01-08,10000.0,10150.0,9940.0,9950.0,158326.0,0.004036,10013.333333,9731.000000,282.333333,545.433333,34.508749
...,...,...,...,...,...,...,...,...,...,...,...,...,...
373184,950130.0,2018-12-21,3805.0,3805.0,3730.0,3765.0,48760.0,-0.010512,3766.666667,4322.250000,555.583333,3265.466667,-11.342602
373185,950130.0,2018-12-24,3835.0,4010.0,3745.0,3875.0,105382.0,0.029216,3876.666667,4315.750000,439.083333,3273.500000,-8.942179
373186,950130.0,2018-12-26,3875.0,3915.0,3700.0,3875.0,105045.0,0.000000,3830.000000,4309.333333,479.333333,3274.966667,-9.757521
373187,950130.0,2018-12-27,4090.0,4090.0,3905.0,3990.0,97302.0,0.029677,3995.000000,4298.750000,303.750000,3289.266667,-6.156387


array([2.0000e+01, 4.0000e+01, 5.0000e+01, ..., 9.0014e+05, 9.5011e+05,
       9.5013e+05])

1561

In [ ]:
len(d_df)
len(s18)
len(s_all)

# 인터넷 코드로 적용해보기. (re_s_all)
* s_all 만들고 M까지 만들고 바로 넘어오기... <br>

근데 간과한 게 있는듯..;;; <br>
무작정 20개씩 묶어서 하면 Code달라도 묶여서 계산되니까..;;; <br>
* 같은 code일 때만 묶어서 구글링 코드 적용시킬것...

In [41]:
re_s_all = s_all.copy()

In [43]:
re_s_all['Code'].unique()
re_s_all['Code'].nunique()

array([    20,     40,     50, ..., 950130, 950140, 950160])

1993

In [77]:
sma = [] # m
adv = [] # d
cci = [] # cci

for i in re_s_all['Code'].unique():
#     re_s_all[re_s_all['Code'] == i]['M']
    tmp_sma = re_s_all[re_s_all['Code'] == i]['M'].rolling(20).mean()
#     tmp_sma[18:20]
    tmp_adv = re_s_all[re_s_all['Code'] == i]['M'].rolling(20).apply(lambda x:pd.Series(x).mad()) # MAD: mean absolute deviation는 평균 절대 편차
#     tmp_adv[18:20]
    tmp_cci = (re_s_all[re_s_all['Code'] == i]['M'] - tmp_sma) / (0.015 * tmp_adv)
#     tmp_cci[18:20]
    sma.extend(list(tmp_sma.values))
    adv.extend(list(tmp_adv.values))
    cci.extend(list(tmp_cci.values))
#     break

len(sma)
len(adv)
len(cci)

# re_s_all

461754

461754

461754

In [79]:
re_s_all_again = pd.concat([re_s_all, pd.Series(sma, name = 'm')], axis = 1)
re_s_all_again = pd.concat([re_s_all_again, pd.Series(adv, name = 'd')], axis = 1)
re_s_all_again = pd.concat([re_s_all_again, pd.Series(cci, name = 'cci')], axis = 1)

In [80]:
re_s_all_again

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,d,cci
0,20,2017-11-01,8850,9170,8840,9170,115335,0.037330,9060.000000,NaN,NaN,NaN
1,20,2017-11-02,9170,9180,9030,9100,68630,-0.007634,9103.333333,NaN,NaN,NaN
2,20,2017-11-03,9080,9270,9070,9240,117255,0.015385,9193.333333,NaN,NaN,NaN
3,20,2017-11-06,9300,9500,9270,9490,369895,0.027056,9420.000000,NaN,NaN,NaN
4,20,2017-11-07,9500,9530,9260,9340,166998,-0.015806,9376.666667,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
461749,950160,2017-12-21,44650,46150,43500,43500,1077534,-0.003436,44383.333333,51690.000000,3787.666667,-128.604535
461750,950160,2017-12-22,44350,45250,43600,44500,816742,0.022989,44450.000000,50799.166667,3620.833333,-116.900652
461751,950160,2017-12-26,44200,46800,44200,45250,960367,0.016854,45416.666667,50136.666667,3433.333333,-91.650485
461752,950160,2017-12-27,46600,49500,46100,49500,1324792,0.093923,48366.666667,49821.666667,3295.000000,-29.438543


In [81]:
re_s_all_again = re_s_all_again[re_s_all_again['Date'].str.contains('2017-')==0]
re_s_all_again

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,d,cci
41,20,2018-01-02,9750,9900,9700,9870,120676,0.012308,9823.333333,9819.666667,268.700000,0.909730
42,20,2018-01-03,9900,10250,9820,10000,268220,0.013171,10023.333333,9795.000000,250.166667,60.848323
43,20,2018-01-04,10050,10050,9680,9750,161342,-0.025000,9826.666667,9763.833333,228.350000,18.344160
44,20,2018-01-05,9750,9980,9750,9910,116604,0.016410,9880.000000,9744.500000,214.816667,42.051362
45,20,2018-01-08,10000,10150,9940,9950,158326,0.004036,10013.333333,9731.000000,205.366667,91.651788
...,...,...,...,...,...,...,...,...,...,...,...,...
461670,950130,2018-12-21,3805,3805,3730,3765,48760,-0.010512,3766.666667,4063.666667,111.900000,-176.943700
461671,950130,2018-12-24,3835,4010,3745,3875,105382,0.029216,3876.666667,4055.083333,119.416667,-99.604559
461672,950130,2018-12-26,3875,3915,3700,3875,105045,0.000000,3830.000000,4044.000000,131.600000,-108.409321
461673,950130,2018-12-27,4090,4090,3905,3990,97302,0.029677,3995.000000,4041.000000,133.500000,-22.971286


In [82]:
# re_s_all_again.to_csv('re_cci_complete.csv', index = False)

In [83]:
pd.read_csv('re_cci_complete.csv')

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,d,cci
0,20,2018-01-02,9750,9900,9700,9870,120676,0.012308,9823.333333,9819.666667,268.700000,0.909730
1,20,2018-01-03,9900,10250,9820,10000,268220,0.013171,10023.333333,9795.000000,250.166667,60.848323
2,20,2018-01-04,10050,10050,9680,9750,161342,-0.025000,9826.666667,9763.833333,228.350000,18.344160
3,20,2018-01-05,9750,9980,9750,9910,116604,0.016410,9880.000000,9744.500000,214.816667,42.051362
4,20,2018-01-08,10000,10150,9940,9950,158326,0.004036,10013.333333,9731.000000,205.366667,91.651788
...,...,...,...,...,...,...,...,...,...,...,...,...
380584,950130,2018-12-21,3805,3805,3730,3765,48760,-0.010512,3766.666667,4063.666667,111.900000,-176.943700
380585,950130,2018-12-24,3835,4010,3745,3875,105382,0.029216,3876.666667,4055.083333,119.416667,-99.604559
380586,950130,2018-12-26,3875,3915,3700,3875,105045,0.000000,3830.000000,4044.000000,131.600000,-108.409321
380587,950130,2018-12-27,4090,4090,3905,3990,97302,0.029677,3995.000000,4041.000000,133.500000,-22.971286


In [39]:
# re_s_all.to_csv('re_cci_with_googling_code.csv', index = False)

In [40]:
pd.read_csv('re_cci_with_googling_code.csv') # no use

,Code,Date,Open,High,Low,Close,Volume,Change,M,SMA,ADV,CCI
0,20,2018-01-02,9750,9900,9700,9870,120676,0.012308,9823.333333,9819.666667,268.700000,0.909730
1,20,2018-01-03,9900,10250,9820,10000,268220,0.013171,10023.333333,9795.000000,250.166667,60.848323
2,20,2018-01-04,10050,10050,9680,9750,161342,-0.025000,9826.666667,9763.833333,228.350000,18.344160
3,20,2018-01-05,9750,9980,9750,9910,116604,0.016410,9880.000000,9744.500000,214.816667,42.051362
4,20,2018-01-08,10000,10150,9940,9950,158326,0.004036,10013.333333,9731.000000,205.366667,91.651788
...,...,...,...,...,...,...,...,...,...,...,...,...
380584,950130,2018-12-21,3805,3805,3730,3765,48760,-0.010512,3766.666667,4063.666667,111.900000,-176.943700
380585,950130,2018-12-24,3835,4010,3745,3875,105382,0.029216,3876.666667,4055.083333,119.416667,-99.604559
380586,950130,2018-12-26,3875,3915,3700,3875,105045,0.000000,3830.000000,4044.000000,131.600000,-108.409321
380587,950130,2018-12-27,4090,4090,3905,3990,97302,0.029677,3995.000000,4041.000000,133.500000,-22.971286


In [29]:
(d_df['Date'].str.contains('2018-')==1).sum()

373189

In [24]:
# d_df.to_csv('CCI_all18_with_Vol&Chng.csv', index = False)

In [25]:
pd.read_csv('CCI_all18_with_Vol&Chng.csv')

,Code,Date,Open,High,Low,Close,Volume,Change,M,m,abs(M-m),d,CCI
0,20,2018-01-02,9750,9900,9700,9870,120676,0.012308,9823.333333,9819.666667,3.666667,566.708333,0.431341
1,20,2018-01-03,9900,10250,9820,10000,268220,0.013171,10023.333333,9795.000000,228.333333,574.341667,26.503775
2,20,2018-01-04,10050,10050,9680,9750,161342,-0.025000,9826.666667,9763.833333,62.833333,567.791667,7.377510
3,20,2018-01-05,9750,9980,9750,9910,116604,0.016410,9880.000000,9744.500000,135.500000,553.483333,16.320877
4,20,2018-01-08,10000,10150,9940,9950,158326,0.004036,10013.333333,9731.000000,282.333333,545.433333,34.508749
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380546,950130,2018-12-21,3805,3805,3730,3765,48760,-0.010512,3766.666667,4063.666667,297.000000,124.979167,-158.426404
380547,950130,2018-12-24,3835,4010,3745,3875,105382,0.029216,3876.666667,4055.083333,178.416667,131.625000,-90.366150
380548,950130,2018-12-26,3875,3915,3700,3875,105045,0.000000,3830.000000,4044.000000,214.000000,139.762500,-102.077930
380549,950130,2018-12-27,4090,4090,3905,3990,97302,0.029677,3995.000000,4041.000000,46.000000,139.479167,-21.986557
